In [ ]:
!pip install awswrangler
!pip install xgboost
!pip install lightgbm
!pip install optuna
!pip install bayesian-optimization
!pip install scikit-optimize

In [1]:
import pandas as pd
import numpy as np
import boto3
from sklearn.feature_selection import RFECV
from xgboost import XGBClassifier
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import make_scorer, precision_score
import pandas as pd
import xgboost as xgb
from sklearn.metrics import classification_report
import awswrangler as wr
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import TimeSeriesSplit, cross_val_score




import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)

In [1]:
import pandas as pd
def get_results(df,prob_col,file_name):
    thresholds = [0.5, 0.51, 0.52, 0.53, 0.54, 0.55, 0.56, 0.57, 0.58, 0.59, 
 0.6, 0.61, 0.62, 0.63, 0.64, 0.65, 0.66, 0.67, 0.68, 0.69, 
 0.7, 0.71, 0.72, 0.73, 0.74, 0.75, 0.76, 0.77, 0.78, 0.79, 
 0.8, 0.81, 0.82, 0.83, 0.84, 0.85, 0.86, 0.87, 0.88, 0.89, 
 0.9, 0.91, 0.92, 0.93, 0.94, 0.95, 0.96, 0.97, 0.98, 0.99, 1.0]
    models = [prob_col]
    tgt_col = 'target_low_intraday'

    results = {model: {top: [] for top in range(1, 11)} for model in models}

    def perform_analysis(df, mod, thr, top):
        unique_days = df['date'].nunique()
        sub_df = df[df[mod] >= thr]
        sub_df.sort_values(by=mod, ascending=False, inplace=True)

        grouped = sub_df.groupby('date').head(top)

        value_counts = grouped[tgt_col].value_counts(normalize=True).round(2).to_dict()
        unique_dates = grouped['date'].nunique()
        total_symbols = grouped[tgt_col].shape[0]
        unique_symbols = grouped['nse_symbol'].nunique()
#         avg_return_2 = round(grouped['next_day_return_high'].mean(), 2)
        days_prct =  round(unique_dates/unique_days,2)

        result = {
            'Threshold': thr,
            'Value Counts': value_counts,
            'No. days preds given': unique_dates,
            'Total Days': unique_days,
            'Total Symbols': total_symbols,
            'Unique Symbols': unique_symbols,
            '%days': days_prct,
#             'Average Return 2%': avg_return_2
        }

        return result, grouped

    for mod in models:
        for thr in thresholds:
            for top in range(1,11):
                result, top_df = perform_analysis(df, mod, thr, top)
                results[mod][top].append(result)
                print(f'{mod} with threshold {thr} for top {top} done')

    # Save results to Excel
    with pd.ExcelWriter(file_name) as writer:
        for mod, tops in results.items():
            for top, mod_results in tops.items():
                summary_df = pd.DataFrame(mod_results)
                sheet_name = f"{mod}_top{top}"
                summary_df.to_excel(writer, sheet_name=sheet_name, index=False)


In [4]:
data = wr.s3.read_parquet(f's3://sg-ds-project/alpha-generator/training_data_transform/intraday_short_model_data_2005_20240ct.parquet')
data.shape

(1573618, 552)

In [4]:
data.date.min(),data.date.max()

(Timestamp('2005-01-03 00:00:00'), Timestamp('2024-10-31 00:00:00'))

In [5]:
data.head(1)

date    open    high     low   close   volume nse_symbol  adj_close  \
0 2020-03-04  361.25  369.75  357.01  362.94  77636.0     360ONE     362.94   

   next_day_return_close  target_close_intraday  next_day_return_high  \
0                   0.72                      0                  5.78   

   target_high_intraday  next_day_return_low  target_low_intraday  \
0                     1                -0.85                    0   

   5d_return_high  target_high_5d  target_high_5d_multiclass  ABER_ATR_29_15  \
0        5.779382             1.0                        1.0        18.51936   

   aber_band_width  aber_momentum  accband_band_width  norm_ad_position  \
0         37.03872       -0.00999           71.332465               0.0   

   alma_momentum  alma_slope  AMATr_LR_24_25_28  AMATr_SR_24_25_28   AO_8_28  \
0       6.258649    5.050155                  1                  0  11.79759   

     ao_roc  ao_zero_cross  AOBV_LR_2  AOBV_SR_2  APO_10_18  apo_roc  \
0 -6.284375              0          0          1  -2.246889   -4.005   

   apo_signal    ATRe_28  atr_pct_close  atr_slope  BBB_21_2.0  BBP_21_2.0  \
0           0  17.789556       4.901514  -0.187021   22.360926    0.366666   

   BIAS_SMA_27  absolute_bias      AR_25  ar_br_ratio  CDL_2CROWS  \
0    -0.002275       0.002275  75.091965     0.613218         0.0   

   CDL_3BLACKCROWS  CDL_3INSIDE  CDL_3LINESTRIKE  CDL_3OUTSIDE  \
0              0.0          0.0              0.0           0.0   

   CDL_3STARSINSOUTH  CDL_3WHITESOLDIERS  CDL_ABANDONEDBABY  CDL_ADVANCEBLOCK  \
0                0.0                 0.0                0.0               0.0   

   CDL_BELTHOLD  CDL_BREAKAWAY  CDL_CLOSINGMARUBOZU  CDL_CONCEALBABYSWALL  \
0           0.0            0.0                  0.0                   0.0   

   CDL_COUNTERATTACK  CDL_DARKCLOUDCOVER  CDL_DOJI_10_0.1  CDL_DOJISTAR  \
0                0.0                 0.0            100.0           0.0   

   CDL_DRAGONFLYDOJI  CDL_ENGULFING  CDL_EVENINGDOJISTAR  CDL_EVENINGSTAR  \
0                0.0            0.0                  0.0              0.0   

   CDL_GAPSIDESIDEWHITE  CDL_GRAVESTONEDOJI  CDL_HAMMER  CDL_HANGINGMAN  \
0                   0.0                 0.0         0.0             0.0   

   CDL_HARAMI  CDL_HARAMICROSS  CDL_HIGHWAVE  CDL_HIKKAKE  CDL_HIKKAKEMOD  \
0         0.0              0.0         100.0          0.0             0.0   

   CDL_HOMINGPIGEON  CDL_IDENTICAL3CROWS  CDL_INNECK  CDL_INSIDE  \
0               0.0                  0.0         0.0           0   

   CDL_INVERTEDHAMMER  CDL_KICKING  CDL_KICKINGBYLENGTH  CDL_LADDERBOTTOM  \
0                 0.0          0.0                  0.0               0.0   

   CDL_LONGLEGGEDDOJI  CDL_LONGLINE  CDL_MARUBOZU  CDL_MATCHINGLOW  \
0               100.0           0.0           0.0              0.0   

   CDL_MATHOLD  CDL_MORNINGDOJISTAR  CDL_MORNINGSTAR  CDL_ONNECK  \
0          0.0                  0.0              0.0         0.0   

   CDL_PIERCING  CDL_RICKSHAWMAN  CDL_RISEFALL3METHODS  CDL_SEPARATINGLINES  \
0           0.0            100.0                   0.0                  0.0   

   CDL_SHOOTINGSTAR  CDL_SHORTLINE  CDL_SPINNINGTOP  CDL_STALLEDPATTERN  \
0               0.0            0.0            100.0                 0.0   

   CDL_STICKSANDWICH  CDL_TAKURI  CDL_TASUKIGAP  CDL_THRUSTING  CDL_TRISTAR  \
0                0.0         0.0            0.0            0.0          0.0   

   CDL_UNIQUE3RIVER  CDL_UPSIDEGAP2CROWS  CDL_XSIDEGAP3METHODS    CMF_22  \
0               0.0                  0.0                   0.0 -0.030499   

   cmf_crossover   cmf_lag  cmf_ma_5      CG_8  cg_cross    cg_lag   cg_ma_5  \
0              0 -0.012527  0.009677 -4.561941         0 -4.576678 -4.562252   

   chop_signal  chop_ma_5  cksp_stop_diff  cdksp_stop_signal    CMO_22  \
0            1  44.445809        36.98819                  0  8.154661   

   cmo_directional_signal  cmo_change  COPC_10_26_4  DEC_25  dema_close_diff  \
0   

In [6]:
data.columns.to_list()

['date',
 'open',
 'high',
 'low',
 'close',
 'volume',
 'nse_symbol',
 'adj_close',
 'next_day_return_close',
 'target_close_intraday',
 'next_day_return_high',
 'target_high_intraday',
 'next_day_return_low',
 'target_low_intraday',
 '5d_return_high',
 'target_high_5d',
 'target_high_5d_multiclass',
 'ABER_ATR_29_15',
 'aber_band_width',
 'aber_momentum',
 'accband_band_width',
 'norm_ad_position',
 'alma_momentum',
 'alma_slope',
 'AMATr_LR_24_25_28',
 'AMATr_SR_24_25_28',
 'AO_8_28',
 'ao_roc',
 'ao_zero_cross',
 'AOBV_LR_2',
 'AOBV_SR_2',
 'APO_10_18',
 'apo_roc',
 'apo_signal',
 'ATRe_28',
 'atr_pct_close',
 'atr_slope',
 'BBB_21_2.0',
 'BBP_21_2.0',
 'BIAS_SMA_27',
 'absolute_bias',
 'AR_25',
 'ar_br_ratio',
 'CDL_2CROWS',
 'CDL_3BLACKCROWS',
 'CDL_3INSIDE',
 'CDL_3LINESTRIKE',
 'CDL_3OUTSIDE',
 'CDL_3STARSINSOUTH',
 'CDL_3WHITESOLDIERS',
 'CDL_ABANDONEDBABY',
 'CDL_ADVANCEBLOCK',
 'CDL_BELTHOLD',
 'CDL_BREAKAWAY',
 'CDL_CLOSINGMARUBOZU',
 'CDL_CONCEALBABYSWALL',
 'CDL_COUNTERAT

In [7]:
features = ['ABER_ATR_29_15',
 'aber_band_width',
 'aber_momentum',
 'accband_band_width',
 'norm_ad_position',
 'alma_momentum',
 'alma_slope',
 'AMATr_LR_24_25_28',
 'AMATr_SR_24_25_28',
 'AO_8_28',
 'ao_roc',
 'ao_zero_cross',
 'AOBV_LR_2',
 'AOBV_SR_2',
 'APO_10_18',
 'apo_roc',
 'apo_signal',
 'ATRe_28',
 'atr_pct_close',
 'atr_slope',
 'BBB_21_2.0',
 'BBP_21_2.0',
 'BIAS_SMA_27',
 'absolute_bias',
 'AR_25',
 'ar_br_ratio',
 'CDL_2CROWS',
 'CDL_3BLACKCROWS',
 'CDL_3INSIDE',
 'CDL_3LINESTRIKE',
 'CDL_3OUTSIDE',
 'CDL_3STARSINSOUTH',
 'CDL_3WHITESOLDIERS',
 'CDL_ABANDONEDBABY',
 'CDL_ADVANCEBLOCK',
 'CDL_BELTHOLD',
 'CDL_BREAKAWAY',
 'CDL_CLOSINGMARUBOZU',
 'CDL_CONCEALBABYSWALL',
 'CDL_COUNTERATTACK',
 'CDL_DARKCLOUDCOVER',
 'CDL_DOJI_10_0.1',
 'CDL_DOJISTAR',
 'CDL_DRAGONFLYDOJI',
 'CDL_ENGULFING',
 'CDL_EVENINGDOJISTAR',
 'CDL_EVENINGSTAR',
 'CDL_GAPSIDESIDEWHITE',
 'CDL_GRAVESTONEDOJI',
 'CDL_HAMMER',
 'CDL_HANGINGMAN',
 'CDL_HARAMI',
 'CDL_HARAMICROSS',
 'CDL_HIGHWAVE',
 'CDL_HIKKAKE',
 'CDL_HIKKAKEMOD',
 'CDL_HOMINGPIGEON',
 'CDL_IDENTICAL3CROWS',
 'CDL_INNECK',
 'CDL_INSIDE',
 'CDL_INVERTEDHAMMER',
 'CDL_KICKING',
 'CDL_KICKINGBYLENGTH',
 'CDL_LADDERBOTTOM',
 'CDL_LONGLEGGEDDOJI',
 'CDL_LONGLINE',
 'CDL_MARUBOZU',
 'CDL_MATCHINGLOW',
 'CDL_MATHOLD',
 'CDL_MORNINGDOJISTAR',
 'CDL_MORNINGSTAR',
 'CDL_ONNECK',
 'CDL_PIERCING',
 'CDL_RICKSHAWMAN',
 'CDL_RISEFALL3METHODS',
 'CDL_SEPARATINGLINES',
 'CDL_SHOOTINGSTAR',
 'CDL_SHORTLINE',
 'CDL_SPINNINGTOP',
 'CDL_STALLEDPATTERN',
 'CDL_STICKSANDWICH',
 'CDL_TAKURI',
 'CDL_TASUKIGAP',
 'CDL_THRUSTING',
 'CDL_TRISTAR',
 'CDL_UNIQUE3RIVER',
 'CDL_UPSIDEGAP2CROWS',
 'CDL_XSIDEGAP3METHODS',
 'CMF_22',
 'cmf_crossover',
 'cmf_lag',
 'cmf_ma_5',
 'CG_8',
 'cg_cross',
 'cg_lag',
 'cg_ma_5',
 'chop_signal',
 'chop_ma_5',
 'cksp_stop_diff',
 'cdksp_stop_signal',
 'CMO_22',
 'cmo_directional_signal',
 'cmo_change',
 'COPC_10_26_4',
 'DEC_25',
 'dema_close_diff',
 'dema_roc',
 'dema_crossover',
 'dm_ratio',
 'dm_diff',
 'donchain_price_relative',
 'donchain_breakout_signal',
 'efi_direction',
 'efi_cross_up',
 'efi_cross_down',
 'ema_roc',
 'price_ema_diff',
 'ENTP_27',
 'entropy_smooth',
 'entropy_change_rate',
 'entropy_above_average',
 'eom_signal',
 'eom_crossover',
 'ER_21',
 'er_signal',
 'er_smooth',
 'BULLP_25',
 'BEARP_25',
 'bull_bear_ratio',
 'bull_power_ema',
 'bear_power_ema',
 'FISHERT_27_5',
 'FISHERTs_27_5',
 'fisher_signal',
 'fisher_smooth',
 'fwma_momentum',
 'fwma_signal',
 'hilo_cross_down',
 'hl2_roc',
 'hlc3_roc',
 'hma_roc',
 'ichimoku_span_diff',
 'ichimoku_tenkan_kijun_cross',
 'INC_25',
 'INERTIA_5_4',
 'inertia_trend',
 'normalized_inertia',
 'trend_reversal',
 'kc_price_above_basis',
 'kc_width',
 'kc_breakout_up',
 'kc_breakout_down',
 'K_27_27',
 'D_27_27',
 'J_27_27',
 'kdj_trend_strength',
 'kdj_crossover',
 'kdj_cross_up',
 'kdj_cross_down',
 'kdj_overbought',
 'kdj_oversold',
 'kst_crossover',
 'KURT_25',
 'kurtosis_change',
 'high_kurtosis',
 'LRa_25',
 'LRm_25',
 'tsf_buy_signal',
 'tsf_sell_signal',
 'LOGRET_27',
 'log_return_volatility',
 'MAD_22',
 'normalized_mad',
 'mad_signal',
 'MASSI_5_28',
 'massi_roc',
 'massi_ma',
 'normalized_massi',
 'roc_rolling_median',
 'normalized_median',
 'above_median',
 'NATR_5',
 'natr_smoothed',
 'natr_normalized',
 'natr_cross_threshold',
 'ohlc4_roc',
 'ohlc4_ma_ratio',
 'pdist_ma_ratio',
 'PCTRET_27',
 'pct_return_smoothed',
 'PGO_26',
 'pgo_ema',
 'pgo_sma',
 'PPO_9_28_24',
 'PPOh_9_28_24',
 'PPOs_9_28_24',
 'ppo_cross',
 'ppo_histogram_strength',
 'PSARaf_0.02_0.2',
 'PSARr_0.02_0.2',
 'PSL_6',
 'PVO_3_29_26',
 'PVOh_3_29_26',
 'PVOs_3_29_26',
 'pvo_cross',
 'pvo_histogram_strength',
 'pvol_divergence',
 'PVR',
 'pvr_change',
 'pvr_buy_signal',
 'pvr_sell_signal',
 'pvt_increasing',
 'pvt_decreasing',
 'pwma_change',
 'pwma_to_sma_ratio',
 'QQE_18_26_4.236',
 'QQE_18_26_4.236_RSIMA',
 'qqe_trend',
 'qqe_band_strength',
 'QS_25',
 'qstick_trend',
 'qstick_strength',
 'RSX_25',
 'rsx_trend',
 'rsx_roc',
 'rsx_signal',
 'RVGI_20_11',
 'RVGIs_20_11',
 'rvgi_trend',
 'rvgi_signal',
 'rvgi_roc',
 'RVI_15',
 'rvi_signal',
 'rvi_divergence',
 'rvi_cross_signal',
 'sima_signal',
 'sima_divergence',
 'sima_cross_signal',
 'SKEW_28',
 'skew_signal',
 'skew_change',
 'extreme_skew',
 'SLOPE_25',
 'slope_signal',
 'slope_change',
 'slope_angle',
 'SQZPRO_9_2.0_15_2_1.5_1',
 'SQZPRO_ON_WIDE',
 'SQZPRO_ON_NORMAL',
 'SQZPRO_ON_NARROW',
 'SQZPRO_OFF',
 'SQZPRO_NO',
 'squeeze_state',
 'sqzpro_momentum',
 'squeeze_duration',
 'ssf_roc',
 'STDEV_27',
 'stdev_momentum',
 'stdev_vs_mean',
 'STOCHRSIk_24_25_15_7',
 'STOCHRSId_24_25_15_7',
 'stochrsi_k_slope',
 'stochrsi_d_slope',
 'stochrsi_overbought',
 'stochrsi_oversold',
 'stochrsi_k_d_crossover_up',
 'stochrsi_k_d_crossover_down',
 'stochrsi_k__diff',
 'SUPERTd_13_3.0',
 'swma_cross_up',
 'swma_cross_down',
 'swma_distance',
 'swma_rate_of_change',
 't3_cross_up',
 't3_cross_down',
 'TEMA_diff',
 'TEMA_divergence',
 'THERMO_22_14.0_8.0',
 'THERMOma_22_14.0_8.0',
 'THERMOl_22_14.0_8.0',
 'THERMOs_22_14.0_8.0',
 'thermo_volatility_ratio',
 'thermo_momentum',
 'thermo_cross_long',
 'thermo_cross_short',
 'TRIMA_diff',
 'TRIMA_divergence',
 'TRIMA_cross_up',
 'TRIMA_cross_down',
 'TRIX_27_5',
 'TRIXs_27_5',
 'TRIX_momentum',
 'TRIX_cross_up',
 'TRIX_cross_down',
 'TRUERANGE_1',
 'TRUE_RANGE_slope',
 'TRUE_RANGE_roll_mean',
 'TRUE_RANGE_pct',
 'TSI_27_30_27',
 'TSIs_27_30_27',
 'TSI_momentum',
 'TSI_overbought',
 'TSI_oversold',
 'TSI_cross_up',
 'TSI_cross_down',
 'TTM_TRND_22',
 'TTM_Cross_Up',
 'TTM_Cross_Down',
 'TTM_Strength',
 'UO_19_15_15',
 'UO_Momentum',
 'UO_Divergence_pct_change',
 'VHF_Trending',
 'VHF_Ranging',
 'VHF_Smoothed',
 'VIDYA_ROC',
 'VIDYA_Crossover',
 'VIDYA_Crossunder',
 'VTXP_21',
 'VIP_Crossover',
 'VIM_Crossover',
 'Vortex_Diff',
 'VIP_Rolling_Sum',
 'VIM_Rolling_Sum',
 'WCP_ROC',
 'WCP_Close_Diff',
 'WILLR_27',
 'rolling_mean_willr',
 'rolling_std_willr',
 'roc',
 'willr_momentum',
 'zlma_cross',
 'zlma_volatility',
 'zlma_volatility_std',
 'zlma_roc',
 'zscore_rolling_avg',
 'RSI_22',
 '1w_avg_RSI_22',
 '1m_avg_RSI_22',
 '1w_slope_RSI_22',
 '1m_slope_RSI_22',
 'RSI_Overbought',
 'RSI_Oversold',
 'RSI_Above_70_5D_Consecutive',
 'RSI_Below_30_5D_Consecutive',
 'RSI_ROC',
 'RSI_Upper_Band',
 'RSI_Lower_Band',
 'divergence_SMA_13_42',
 'convergence_SMA_13_42',
 'ratio_adj_close_SMA13',
 'diff_SMA_13_42',
 'days_since_divergence_SMA_13_42',
 'days_since_convergence_SMA_13_42',
 'ratio_adj_close_SMA_42',
 'divergence_SMA_50_100',
 'convergence_SMA_50_100',
 'ratio_adj_close_SMA50',
 'diff_SMA_50_100',
 'days_since_divergence_SMA_50_100',
 'days_since_convergence_SMA_50_100',
 'ratio_adj_close_SMA_100',
 'divergence_EMA_8_42',
 'convergence_EMA_8_42',
 'ratio_EMA_8',
 'cv_EMA_8',
 'diff_ema_8_42',
 'diff_ema_8_42_pct',
 'days_since_divergence_EMA_8_42',
 'days_since_convergence_EMA_8_42',
 'ratio_EMA_42',
 'cv_EMA_42',
 'divergence_EMA_50_100',
 'convergence_EMA_50_100',
 'ratio_EMA_50',
 'cv_EMA_50',
 'diff_ema_50_100',
 'diff_ema_50_100_pct',
 'days_since_divergence_EMA_50_100',
 'days_since_convergence_EMA_50_100',
 'ratio_EMA_100',
 'cv_EMA_100',
 'divergence_DEMA_18_38',
 'convergence_DEMA_18_38',
 'ratio_DEMA_18',
 'cv_DEMA_18',
 'diff_DEMA_18_38',
 'days_since_divergence_DEMA_18_38',
 'days_since_convergence_DEMA_18_38',
 'ratio_DEMA_38',
 'cv_DEMA_38',
 'divergence_DEMA_50_100',
 'convergence_DEMA_50_100',
 'ratio_DEMA_50',
 'cv_DEMA_50',
 'days_since_divergence_DEMA_50_100',
 'days_since_convergence_DEMA_50_100',
 'BB_deviation_upper_band_19',
 'price_within_BBANDS',
 'squeeze_BBANDS',
 'expansion_BBANDS',
 'above_ub_BBANDS',
 'below_lb_BBANDS',
 'dist_frm_ub_BBANDS',
 'dist_frm_ub_BBANDS_pct',
 'dist_frm_lb_BBANDS',
 'dist_frm_lb_BBANDS_pct',
 '1w_slope_ub_BBANDS',
 '1m_slope_ub_BBANDS',
 '25_CCI',
 'CCI_Overbought',
 'CCI_Oversold',
 'CCI_ROC',
 'CCI_1w_slope',
 'Bullish_Trend_CCI',
 'Bearish_Trend_CCI',
 'CCI_Range',
 'MACD_histogram_21_28',
 'convergence_MACD_21_28',
 'divergence_MACD_21_28',
 'MACD_Above_Signal',
 'MACD_Below_Signal',
 'MACD_Histogram_Volatility',
 'MACD_Bullish_Zone',
 'MACD_Bearish_Zone',
 'MACD_ROC',
 'MACD_Slope',
 'MACD_Cross_Above_0',
 'MACD_Cross_Below_0',
 'MACD_Absolute',
 'mfi25_MFI',
 'MFI_Overbought',
 'MFI_Oversold',
 '1w_slope_MFI',
 '1m_slope_MFI',
 'divergence_MFI',
 '27_ROC',
 '%K_STOCH',
 '%D_STOCH',
 '%K_%D_diff_STOCH',
 'STOCH_Overbought_Cross',
 'STOCH_Oversold_Cross',
 'Stoch_ROC',
 'Stoch_Slope',
 'STOCH_Bullish_Trend',
 'STOCH_Bearish_Trend',
 'Stoch_OSC_Range',
 'VWAP_Deviation_7',
 'log_OBV_5',
 'ADX_22',
 'ADX_plusDI_22',
 'ADX_minusDI_22',
 'ADX_diff_plusDI_minusDI_22',
 'ADX_Strong_Trend',
 'ADX_Weak_Trend',
 'ADX_ROC',
 'ADX_Above_Threshold',
 'ADX_Below_Threshold',
 '1w_slope_ADX_22',
 '1m_slope_ADX_22',
 'BOP',
 'KAMA_difference_7_27',
 'KAMA_difference_7_27_pct',
 'days_since_divergence_KAMA_7_27',
 'days_since_convergence_KAMA_7_27',
 'KAMA_difference_50_100_pct',
 'diff_adj_close_SAR',
 'diff_adj_close_SAR_pct',
 'AROON_Oscillator27',
 'AROON_Oscillator27_5d_uptrend_count',
 'AROON_Oscillator27_5d_downtrend_count',
 'Doji',
 'Engulfing',
 'Harami',
 'Inverted_Hammer',
 'Morning_Star',
 'Evening_Star',
 'Marubozu',
 'Inside_Bar_Pattern',
 'BullishHorn',
 'BearHorn',
 'BullishHigh',
 'BearHigh',
 'BullishLow',
 'BearLow',
 'pct_awy_52w_high',
 'pct_awy_6m_high',
 'pct_awy_3m_high',
 'pct_awy_1m_high',
 'pct_awy_1w_high',
 'pct_awy_52w_low',
 'pct_awy_6m_low',
 'pct_awy_3m_low',
 'pct_awy_1m_low',
 'pct_awy_1w_low',
 '1d_gap_up',
 '1d_gap_down',
 '1m_gap_up_cnts',
 '1m_gap_down_cnts',
 '1w_gap_up_cnts',
 '1w_gap_down_cnts',
 '52w_vol',
 '6m_vol',
 '3m_vol',
 '1m_vol',
 '1w_vol',
 'momentum_1w',
 'momentum_1m',
 '1w_vol_EMA_14',
 '1m_vol_EMA_14',
 '3m_vol_EMA_14',
 '6m_vol_EMA_14',
 '1y_vol_EMA_14',
 'daily_vol_chng_pct',
 'weekly_vol_chng_pct',
 'monthly_vol_chng_pct',
 'CMF_1w',
 'CMF_1m',
 'VRSI_1w',
 'VRSI_1m',
 'TR',
 'pct_awy_PP_5',
 'pct_awy_R1_5',
 'pct_awy_R2_5',
 'pct_awy_R3_5',
 'pct_awy_S1_5',
 'pct_awy_S2_5',
 'pct_awy_S3_5',
 'pct_awy_PP_20',
 'pct_awy_R1_20',
 'pct_awy_R2_20',
 'pct_awy_R3_20',
 'pct_awy_S1_20',
 'pct_awy_S2_20',
 'pct_awy_S3_20']

In [8]:
train = data[(data['date']>='2015-01-01') & (data['date']<'2024-01-01')]
test = data[(data['date']>='2024-01-04') & (data['date']<'2024-10-01')]
oot = data[data['date']>='2024-10-01']

In [9]:
tgt = 'target_low_intraday'
X_train = train[features]
y_train = train[tgt]
X_test = test[features]
y_test = test[tgt]

In [10]:
X_train.shape,y_train.shape,X_test.shape,y_test.shape,oot.shape

((832420, 535), (832420,), (83826, 535), (83826,), (10274, 552))

In [11]:
train['target_low_intraday'].value_counts(normalize=True).round(2)

0    0.66
1    0.34
Name: target_low_intraday, dtype: Float64

In [ ]:
# import pandas as pd
# from scipy.stats import spearmanr


# features = features

# spearman_corr = {}
# for feature in features:
#     corr, p_value = spearmanr(train[feature], train[tgt])
#     spearman_corr[feature] = (corr, p_value)

# spearman_df = pd.DataFrame.from_dict(spearman_corr, orient='index', columns=['Spearman Correlation', 'P-Value'])

# spearman_df = spearman_df.sort_values(by='Spearman Correlation', ascending=False)

# significant_features = spearman_df[spearman_df['P-Value'] < 0.05]

In [ ]:
# significant_features

In [ ]:
# len(significant_features)

In [ ]:
# significant_features = significant_features.reset_index()['index'].tolist()

In [ ]:
# significant_features

In [12]:
significant_features = ['NATR_5',
 'TRUE_RANGE_pct',
 'natr_smoothed',
 'atr_pct_close',
 'cv_EMA_8',
 'pct_awy_S3_5',
 'cv_DEMA_18',
 'BBB_21_2.0',
 'pct_awy_S3_20',
 'stdev_vs_mean',
 'cv_DEMA_38',
 'cv_EMA_42',
 'normalized_mad',
 'cv_DEMA_50',
 'cv_EMA_50',
 'cv_EMA_100',
 'log_return_volatility',
 'pct_awy_S2_5',
 'absolute_bias',
 'pct_awy_S2_20',
 'pct_awy_1w_high',
 'natr_normalized',
 'ppo_histogram_strength',
 'atr_slope',
 'pct_awy_1w_low',
 'pdist_ma_ratio',
 'pct_awy_1m_low',
 'pct_awy_3m_low',
 'pct_awy_1m_high',
 'MASSI_5_28',
 'chop_signal',
 'pct_awy_S1_5',
 'pct_awy_6m_low',
 'dist_frm_lb_BBANDS_pct',
 'pct_awy_S1_20',
 'thermo_volatility_ratio',
 'pct_awy_52w_low',
 'pct_awy_3m_high',
 'PVO_3_29_26',
 'pct_awy_6m_high',
 'monthly_vol_chng_pct',
 'VRSI_1m',
 'normalized_massi',
 'massi_ma',
 'ADX_22',
 'massi_roc',
 'pct_awy_52w_high',
 'PVOs_3_29_26',
 'above_ub_BBANDS',
 'ER_21',
 'pvo_cross',
 'er_smooth',
 'pvo_histogram_strength',
 'ADX_Above_Threshold',
 'PVOh_3_29_26',
 'weekly_vol_chng_pct',
 'ADX_Strong_Trend',
 'MFI_Overbought',
 'RSI_Above_70_5D_Consecutive',
 'VRSI_1w',
 'SQZPRO_OFF',
 'stdev_momentum',
 'VHF_Trending',
 'er_signal',
 'SQZPRO_NO',
 'VHF_Smoothed',
 'TSI_oversold',
 'ADX_plusDI_22',
 'dist_frm_ub_BBANDS',
 'RSI_Below_30_5D_Consecutive',
 'RSI_Overbought',
 'RSI_Oversold',
 'ADX_ROC',
 'below_lb_BBANDS',
 'qqe_band_strength',
 'TRUE_RANGE_slope',
 '1w_slope_ADX_22',
 'CCI_Oversold',
 'CCI_Overbought',
 'mfi25_MFI',
 'CCI_Range',
 'extreme_skew',
 '1w_slope_ub_BBANDS',
 'TSI_overbought',
 'CMF_1m',
 '1m_slope_ADX_22',
 'thermo_momentum',
 'ADX_minusDI_22',
 'PSARaf_0.02_0.2',
 'daily_vol_chng_pct',
 'high_kurtosis',
 'pvol_divergence',
 'days_since_convergence_EMA_50_100',
 'days_since_divergence_EMA_50_100',
 'QS_25',
 'STOCH_Oversold_Cross',
 'BearLow',
 'thermo_cross_long',
 'THERMOs_22_14.0_8.0',
 'PSARr_0.02_0.2',
 'MFI_Oversold',
 'KURT_25',
 'entropy_above_average',
 'days_since_convergence_EMA_8_42',
 'pvr_buy_signal',
 'COPC_10_26_4',
 'expansion_BBANDS',
 'days_since_convergence_SMA_50_100',
 'ohlc4_ma_ratio',
 'ratio_EMA_100',
 'diff_ema_50_100_pct',
 'ratio_adj_close_SMA50',
 'ratio_EMA_50',
 'days_since_divergence_EMA_8_42',
 'ratio_adj_close_SMA_42',
 'BIAS_SMA_27',
 'ratio_EMA_42',
 'bull_bear_ratio',
 '1m_slope_ub_BBANDS',
 'PPO_9_28_24',
 'AR_25',
 'diff_ema_8_42_pct',
 'STOCH_Overbought_Cross',
 'days_since_divergence_SMA_50_100',
 '27_ROC',
 'ratio_adj_close_SMA_100',
 'PCTRET_27',
 'LOGRET_27',
 'hl2_roc',
 'diff_adj_close_SAR_pct',
 'hlc3_roc',
 'WCP_ROC',
 'CMF_22',
 'CMF_1w',
 'TRIXs_27_5',
 'TRIX_27_5',
 'kdj_oversold',
 'normalized_median',
 'ssf_roc',
 'PPOs_9_28_24',
 'KAMA_difference_7_27_pct',
 'pct_return_smoothed',
 'ADX_diff_plusDI_minusDI_22',
 'qstick_trend',
 'ENTP_27',
 'rolling_std_willr',
 'ao_roc',
 'swma_rate_of_change',
 'entropy_smooth',
 'cmf_crossover',
 'pvt_decreasing',
 'norm_ad_position',
 'cmf_lag',
 'cg_lag',
 'stochrsi_k__diff',
 'cg_ma_5',
 'hma_roc',
 'STOCH_Bearish_Trend',
 'CG_8',
 'dm_ratio',
 'ratio_DEMA_50',
 '1w_slope_MFI',
 'BullishHorn',
 'ratio_adj_close_SMA13',
 'stochrsi_k_d_crossover_up',
 'AOBV_LR_2',
 'pct_awy_PP_20',
 'days_since_convergence_SMA_13_42',
 'days_since_convergence_DEMA_50_100',
 '1d_gap_down',
 '25_CCI',
 'efi_cross_down',
 'THERMO_22_14.0_8.0',
 '1w_gap_down_cnts',
 'SKEW_28',
 'ratio_DEMA_38',
 'rvgi_roc',
 'BullishHigh',
 'days_since_divergence_DEMA_18_38',
 'hilo_cross_down',
 'fisher_signal',
 'trend_reversal',
 'bear_power_ema',
 'PGO_26',
 'stochrsi_oversold',
 'days_since_divergence_SMA_13_42',
 'diff_adj_close_SAR',
 'cmf_ma_5',
 'CDL_HIKKAKE',
 'BBP_21_2.0',
 'MACD_Slope',
 'pvr_sell_signal',
 'TSI_cross_up',
 'WCP_Close_Diff',
 'RSI_Upper_Band',
 'RSI_22',
 'RSI_Lower_Band',
 'TTM_TRND_22',
 'ratio_EMA_8',
 'stochrsi_d_slope',
 'efi_direction',
 'pgo_ema',
 'convergence_DEMA_18_38',
 'cksp_stop_diff',
 'Bullish_Trend_CCI',
 'MACD_Above_Signal',
 '1w_avg_RSI_22',
 'CMO_22',
 'stochrsi_k_slope',
 'MACD_histogram_21_28',
 'sima_cross_signal',
 't3_cross_up',
 'CDL_MORNINGSTAR',
 'fwma_signal',
 'swma_cross_up',
 'TSIs_27_30_27',
 'Morning_Star',
 'pvr_change',
 'TTM_Strength',
 'ratio_DEMA_18',
 'roc',
 'CCI_1w_slope',
 'convergence_MACD_21_28',
 'rsx_roc',
 'convergence_DEMA_50_100',
 'Vortex_Diff',
 '1m_slope_RSI_22',
 'pgo_sma',
 'dema_close_diff',
 'roc_rolling_median',
 'TRIX_cross_up',
 'ar_br_ratio',
 'VIP_Crossover',
 'above_median',
 'kst_crossover',
 'willr_momentum',
 'convergence_EMA_8_42',
 'sqzpro_momentum',
 'CDL_PIERCING',
 'days_since_divergence_DEMA_50_100',
 'rvgi_signal',
 'rvgi_trend',
 'skew_signal',
 'sima_divergence',
 'CDL_MORNINGDOJISTAR',
 'TSI_27_30_27',
 'sima_signal',
 'kdj_cross_up',
 'kdj_crossover',
 'divergence_EMA_50_100',
 'MACD_ROC',
 'ppo_cross',
 'rsx_trend',
 'CDL_SPINNINGTOP',
 'INC_25',
 'slope_signal',
 'kc_breakout_up',
 '1m_avg_RSI_22',
 'CDL_HANGINGMAN',
 'normalized_inertia',
 'divergence_DEMA_18_38',
 'TSI_cross_down',
 'CDL_LADDERBOTTOM',
 'BEARP_25',
 'CDL_HIKKAKEMOD',
 'kdj_trend_strength',
 'CDL_ADVANCEBLOCK',
 'zscore_rolling_avg',
 'SUPERTd_13_3.0',
 'kc_breakout_down',
 'QQE_18_26_4.236_RSIMA',
 'tsf_buy_signal',
 '%D_STOCH',
 'ao_zero_cross',
 'zlma_roc',
 'apo_roc',
 'divergence_DEMA_50_100',
 'MACD_Cross_Above_0',
 'divergence_MACD_21_28',
 'AROON_Oscillator27_5d_downtrend_count',
 'CDL_MATHOLD',
 'CDL_3WHITESOLDIERS',
 'TTM_Cross_Down',
 'RSX_25',
 'KAMA_difference_7_27',
 'CDL_3LINESTRIKE',
 'CDL_DOJISTAR',
 'SQZPRO_9_2.0_15_2_1.5_1',
 'AROON_Oscillator27',
 'MACD_Cross_Below_0',
 'dema_crossover',
 'TRIX_momentum',
 'tsf_sell_signal',
 'CDL_STALLEDPATTERN',
 'CDL_UNIQUE3RIVER',
 'SLOPE_25',
 'slope_angle',
 'VIDYA_Crossunder',
 'APO_10_18',
 'fisher_smooth',
 'pct_awy_PP_5',
 'CDL_THRUSTING',
 'skew_change',
 'qqe_trend',
 'zlma_volatility',
 'DEC_25',
 'VIM_Rolling_Sum',
 'CDL_EVENINGDOJISTAR',
 'TRIMA_diff',
 'CCI_ROC',
 'fwma_momentum',
 'INERTIA_5_4',
 'kdj_cross_down',
 'THERMOl_22_14.0_8.0',
 'kdj_overbought',
 'VIM_Crossover',
 'diff_SMA_50_100',
 'pwma_change',
 'Marubozu',
 'CDL_MARUBOZU',
 'kurtosis_change',
 '1w_vol',
 'rolling_mean_willr',
 'CDL_EVENINGSTAR',
 'KAMA_difference_50_100_pct',
 'CDL_TAKURI',
 'kc_price_above_basis',
 'diff_ema_8_42',
 'Evening_Star',
 'CDL_DRAGONFLYDOJI',
 'diff_SMA_13_42',
 'TEMA_diff',
 '1w_gap_up_cnts',
 'swma_cross_down',
 'slope_change',
 '1d_gap_up',
 'RVGIs_20_11',
 'MACD_Below_Signal',
 'Bearish_Trend_CCI',
 'VWAP_Deviation_7',
 'K_27_27',
 'CDL_CLOSINGMARUBOZU',
 'FISHERTs_27_5',
 'stochrsi_k_d_crossover_down',
 'UO_Divergence_pct_change',
 'dm_diff',
 't3_cross_down',
 'CDL_INSIDE',
 'CDL_BELTHOLD',
 'BULLP_25',
 'STOCHRSId_24_25_15_7',
 'CDL_HAMMER',
 'CDL_GRAVESTONEDOJI',
 'TEMA_divergence',
 'VIDYA_Crossover',
 'RSI_ROC',
 'alma_momentum',
 'CDL_HARAMICROSS',
 'rsx_signal',
 'pwma_to_sma_ratio',
 'VIP_Rolling_Sum',
 'alma_slope',
 'TR',
 '%K_%D_diff_STOCH',
 'PSL_6',
 'TRUERANGE_1',
 'MACD_Bearish_Zone',
 'cmo_change',
 'CDL_HOMINGPIGEON',
 'days_since_convergence_KAMA_7_27',
 'AOBV_SR_2',
 'Stoch_OSC_Range',
 'aber_momentum',
 'VTXP_21',
 'rvi_signal',
 'rvi_cross_signal',
 'BOP',
 'diff_ema_50_100',
 'STOCH_Bullish_Trend',
 'pvt_increasing',
 'D_27_27',
 'Stoch_ROC',
 'rvi_divergence',
 'days_since_divergence_KAMA_7_27',
 'BearHigh',
 'MACD_Absolute',
 'bull_power_ema',
 'Harami',
 'CDL_HARAMI',
 'RVI_15',
 'BullishLow',
 'accband_band_width',
 'kc_width',
 'CDL_LONGLINE',
 'squeeze_BBANDS',
 'CDL_MATCHINGLOW',
 'eom_signal',
 'MAD_22',
 'zlma_volatility_std',
 'divergence_MFI',
 'STDEV_27',
 'CDL_DOJI_10_0.1',
 'log_OBV_5',
 'Doji',
 'CDL_LONGLEGGEDDOJI',
 'CDL_RICKSHAWMAN',
 '1m_vol',
 'qstick_strength',
 'MACD_Histogram_Volatility',
 'dist_frm_lb_BBANDS',
 '1w_vol_EMA_14',
 'TRUE_RANGE_roll_mean',
 'THERMOma_22_14.0_8.0',
 '3m_vol',
 '1m_vol_EMA_14',
 'ABER_ATR_29_15',
 'aber_band_width',
 'ADX_Weak_Trend',
 'VHF_Ranging',
 '6m_vol',
 '3m_vol_EMA_14',
 'ATRe_28',
 '6m_vol_EMA_14',
 'mad_signal',
 'ADX_Below_Threshold',
 '52w_vol',
 '1y_vol_EMA_14',
 'SQZPRO_ON_NARROW',
 'SQZPRO_ON_WIDE',
 'pct_awy_R1_20',
 'SQZPRO_ON_NORMAL',
 'squeeze_state',
 'squeeze_duration',
 'dist_frm_ub_BBANDS_pct',
 'BB_deviation_upper_band_19',
 'price_within_BBANDS',
 'chop_ma_5',
 'pct_awy_R1_5',
 'pct_awy_R2_20',
 'pct_awy_R2_5',
 'pct_awy_R3_20',
 'pct_awy_R3_5']

In [13]:
import lightgbm as lgb
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score
from skopt import BayesSearchCV

In [14]:
train_data = lgb.Dataset(X_train[significant_features], label=y_train)
test_data = lgb.Dataset(X_test[significant_features], label=y_test)

In [15]:
params = {
    'objective': 'binary',
    'metric': 'precision',
    'boosting_type': 'gbdt',
    'verbose': -1
}

# Train the model
default_model = lgb.train(params, train_data, valid_sets=[test_data])
test['prob_base'] = default_model.predict(test[significant_features])
oot['prob_base'] = default_model.predict(oot[significant_features])

In [16]:
default_model

In [17]:
get_results(test,'prob_base','shortcalls_lightgbm_significant_feat_base.xlsx')

prob_base with threshold 0.5 for top 1 done
prob_base with threshold 0.5 for top 2 done
prob_base with threshold 0.5 for top 3 done
prob_base with threshold 0.5 for top 4 done
prob_base with threshold 0.5 for top 5 done
prob_base with threshold 0.5 for top 6 done
prob_base with threshold 0.5 for top 7 done
prob_base with threshold 0.5 for top 8 done
prob_base with threshold 0.5 for top 9 done
prob_base with threshold 0.5 for top 10 done
prob_base with threshold 0.51 for top 1 done
prob_base with threshold 0.51 for top 2 done
prob_base with threshold 0.51 for top 3 done
prob_base with threshold 0.51 for top 4 done
prob_base with threshold 0.51 for top 5 done
prob_base with threshold 0.51 for top 6 done
prob_base with threshold 0.51 for top 7 done
prob_base with threshold 0.51 for top 8 done
prob_base with threshold 0.51 for top 9 done
prob_base with threshold 0.51 for top 10 done
prob_base with threshold 0.52 for top 1 done
prob_base with threshold 0.52 for top 2 done
prob_base with thr

In [37]:
b = test[test['prob_base']>=0.69].sort_values(by=['date','prob_base'],ascending=[True,False])

In [38]:
a = b.groupby('date').head(10)

In [39]:
a[['date','nse_symbol','next_day_return_low','target_low_intraday','prob_base']].tail(30)

date  nse_symbol  next_day_return_low  target_low_intraday  \
1476508 2024-08-13  TRITURBINE                -4.65                    1   
1476509 2024-08-14  TRITURBINE                -3.16                    1   
285711  2024-08-19  CAPLIPOINT                -3.46                    1   
531324  2024-08-19         FSL                -6.15                    1   
577632  2024-08-19  GODFRYPHLP                -3.98                    1   
577633  2024-08-20  GODFRYPHLP                -1.49                    0   
73120   2024-08-20    ANGELONE                -1.61                    0   
1514900 2024-08-22      VARROC                -4.08                    1   
1127698 2024-08-23        PCBL                -3.83                    1   
890151  2024-08-27    KFINTECH                -6.65                    1   
1413539 2024-08-27  TATAINVEST                -4.89                    1   
1122815 2024-08-30       PAYTM                -5.12                    1   
626504  2024-08-30        GSPL                -1.18                    0   
587419  2024-09-03   GODREJIND                -4.89                    1   
299701  2024-09-04         CCL                -4.07                    1   
1200856 2024-09-10  PRSMJOHNSN                -1.82                    0   
1344933 2024-09-10    SUMICHEM                -4.53                    1   
1200857 2024-09-11  PRSMJOHNSN                -5.28                    1   
854983  2024-09-13  JUBLPHARMA                -1.68                    0   
267234  2024-09-16         BSE                -3.07                    1   
577652  2024-09-16  GODFRYPHLP                -5.11                    1   
267236  2024-09-18         BSE                -8.21                    1   
379831  2024-09-20  CONCORDBIO                -5.83                    1   
1253293 2024-09-20       RITES                -3.48                    1   
1275684 2024-09-23        SBFC                -6.25                    1   
834699  2024-09-24  JMFINANCIL                -2.13                    1   
120752  2024-09-24    ASTRAZEN                -2.22                    1   
1270612 2024-09-24    SAPPHIRE                -3.77                    1   
1275501 2024-09-25    SAREGAMA                -1.98                    0   
1275504 2024-09-30    SAREGAMA                -3.31                    1   

         prob_base  
1476508   0.712938  
1476509   0.717391  
285711    0.730740  
531324    0.703592  
577632    0.701325  
577633    0.722587  
73120     0.702996  
1514900   0.700864  
1127698   0.717772  
890151    0.700852  
1413539   0.699830  
1122815   0.706246  
626504    0.691031  
587419    0.716602  
299701    0.695236  
1200856   0.756150  
1344933   0.742896  
1200857   0.717034  
854983    0.714701  
267234    0.706385  
577652    0.705033  
267236    0.692179  
379831    0.713436  
1253293   0.696252  
1275684   0.741934  
834699    0.727285  
120752    0.704432  
1270612   0.690509  
1275501   0.720268  
1275504   0.706732

In [40]:
a['date'] = pd.to_datetime(a['date'])

a['year_month'] = a['date'].dt.to_period('M')

result = a.groupby('year_month').apply(lambda x: pd.Series({
    'target_1_precision': round(x['target_low_intraday'].value_counts(normalize=True).get(1, 0), 2),  # Use built-in round() function
    'tot_nse_symbol': x['nse_symbol'].count(),
    'unique_nse_symbol': x['nse_symbol'].nunique(),
    'unique_days': x['date'].nunique()
})).reset_index()

In [41]:
result

year_month  target_1_precision  tot_nse_symbol  unique_nse_symbol  \
0    2024-01                0.70            53.0               30.0   
1    2024-02                0.74            70.0               31.0   
2    2024-03                0.79            34.0               20.0   
3    2024-04                0.75            12.0               10.0   
4    2024-05                0.82            28.0               16.0   
5    2024-06                0.75            71.0               46.0   
6    2024-07                0.84            31.0               19.0   
7    2024-08                0.67            18.0               13.0   
8    2024-09                0.82            17.0               14.0   

   unique_days  
0         15.0  
1         17.0  
2         14.0  
3          9.0  
4         12.0  
5         18.0  
6         18.0  
7         11.0  
8         12.0

In [42]:
importances = default_model.feature_importance(importance_type='split')  # or 'gain' depending on the type

feature_names = train[significant_features].columns  

importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': importances
})

importance_df = importance_df.sort_values(by='Importance', ascending=False)
importance_df


Feature  Importance
1                           TRUE_RANGE_pct         134
0                                   NATR_5         128
3                            atr_pct_close         118
402                              log_OBV_5          70
231                            ar_br_ratio          59
87                          ADX_minusDI_22          59
47                            PVOs_3_29_26          51
43                                massi_ma          51
198                            ratio_EMA_8          49
24                          pct_awy_1w_low          47
120                                  AR_25          46
36                         pct_awy_52w_low          39
31                            pct_awy_S1_5          39
376                                    BOP          39
355                                RSI_ROC          39
189                             BBP_21_2.0          36
84                                  CMF_1m          36
26                          pct_awy_1m_low          34
436                              chop_ma_5          34
372                          aber_momentum          32
92       days_since_convergence_EMA_50_100          32
18                           absolute_bias          31
67                           ADX_plusDI_22          28
133                                 CMF_1w          28
42                        normalized_massi          28
32                          pct_awy_6m_low          28
259                     normalized_inertia          26
54                            PVOh_3_29_26          26
220                                    roc          25
193                         WCP_Close_Diff          25
367                             cmo_change          24
80                               CCI_Range          23
35                 thermo_volatility_ratio          23
107      days_since_convergence_SMA_50_100          22
363                       %K_%D_diff_STOCH          22
89                      daily_vol_chng_pct          22
146                                 ao_roc          20
55                     weekly_vol_chng_pct          20
153                                 cg_lag          20
353                        TEMA_divergence          20
21                         natr_normalized          20
2                            natr_smoothed          18
75                        TRUE_RANGE_slope          18
155                                cg_ma_5          18
29                              MASSI_5_28          18
59                                 VRSI_1w          17
359                      pwma_to_sma_ratio          16
275                                apo_roc          15
350                   STOCHRSId_24_25_15_7          14
25                          pdist_ma_ratio          13
16                   log_return_volatility          13
381                              Stoch_ROC          13
173                       1w_gap_down_cnts          13
373                                VTXP_21          13
45                               massi_roc          13
93        days_since_divergence_EMA_50_100          13
342                          FISHERTs_27_5          12
441                           pct_awy_R3_5          12
5                             pct_awy_S3_5          12
72                                 ADX_ROC          12
15                              cv_EMA_100          11
203                         cksp_stop_diff          11
300                           pct_awy_PP_5          11
178       days_since_divergence_DEMA_18_38          11
4                                 cv_EMA_8          11
124                                 27_ROC          11
168     days_since_convergence_DEMA_50_100          11
91                         pvol_divergence          11
158                                   CG_8          11
49                                   ER_21          11
27                          pct_awy_3m_low          11
117                        bull_bear_ratio          10
103        days_sinc

In [ ]:
from sklearn.feature_selection import RFECV
from xgboost import XGBClassifier
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import make_scorer, precision_score
import pandas as pd

model = XGBClassifier(objective='binary:logistic', eval_metric='logloss')

precision_scorer = make_scorer(precision_score)
tscv = TimeSeriesSplit(n_splits=5)
rfecv = RFECV(estimator=model, step=20, cv=tscv, scoring=precision_scorer, verbose=3, n_jobs=48)
rfecv.fit(train[significant_features], train[tgt])

In [ ]:
print(f"Optimal number of features: {rfecv.n_features_}")

# Get the selected features
selected_features = train[significant_features].columns[rfecv.support_]
print("Selected Features:", selected_features)

In [ ]:
selected_features = ['NATR_5', 'TRUE_RANGE_pct', 'natr_smoothed', 'atr_pct_close',
       'pct_awy_S3_5', 'absolute_bias', 'pct_awy_1w_low', 'pct_awy_1m_low',
       'pct_awy_S1_5', 'pct_awy_52w_low', 'pvol_divergence', '1d_gap_down',
       'BBP_21_2.0', 'ratio_EMA_8', 'CMO_22', 'ar_br_ratio',
       'THERMOl_22_14.0_8.0', '1d_gap_up', 'RSI_ROC', 'cmo_change', 'BOP',
       'price_within_BBANDS']

In [ ]:
param_grid = {
    'max_depth': (4, 20),            # Maximum depth of the tree
    'learning_rate': (0.01, 0.1),    # Learning rate
    'n_estimators': (50, 500),       # Number of boosting rounds
    'num_leaves': (20, 150),         # Maximum number of leaves per tree
    'min_child_weight': (1, 20),     # Minimum sum of instance weight (hessian) in a child
    'subsample': (0.1, 0.9),         # Subsample ratio of the training data
    'colsample_bytree': (0.1, 0.9),  # Subsample ratio of features
    'reg_lambda': (0.1, 10),         # L2 regularization
    'reg_alpha': (0.03, 10),         # L1 regularization
    'scale_pos_weight': (1, 20)      # Adjust for class imbalance
}


In [ ]:
from sklearn.model_selection import TimeSeriesSplit, cross_val_score
from lightgbm import LGBMClassifier
from bayes_opt import BayesianOptimization
from sklearn.metrics import make_scorer, f1_score
import numpy as np


def lgb_evaluate(max_depth, learning_rate, n_estimators, num_leaves, min_child_weight, subsample, colsample_bytree, reg_lambda, reg_alpha, scale_pos_weight):
    params = {
        'n_estimators': int(n_estimators),  
        'max_depth': int(max_depth),       
        'learning_rate': learning_rate,     
        'num_leaves': int(num_leaves),      
        'min_child_weight': min_child_weight,  
        'subsample': subsample,             
        'colsample_bytree': colsample_bytree,  
        'reg_lambda': reg_lambda,           
        'reg_alpha': reg_alpha,             
        'scale_pos_weight': scale_pos_weight,  
        'objective': 'binary',              
        'random_state': 42,
        'n_jobs': 47  
    }
    
    tuned_model = LGBMClassifier(**params)
    
    tscv = TimeSeriesSplit(n_splits=5)
    
    scores = cross_val_score(
        tuned_model, 
        train[selected_features], 
        y_train, 
        cv=tscv, 
        scoring='precision',  
        verbose=1
    )
    
    return np.mean(scores)  


pbounds_lgb = {
    'max_depth': (4, 50),            # Maximum depth of the tree
    'learning_rate': (0.01, 0.3),    # Learning rate
    'n_estimators': (50, 500),       # Number of boosting rounds
    'num_leaves': (20, 150),         # Maximum number of leaves per tree
    'min_child_weight': (1, 50),     # Minimum sum of instance weight (hessian) in a child
    'subsample': (0.1, 0.9),         # Subsample ratio of the training data
    'colsample_bytree': (0.1, 0.9),  # Subsample ratio of features
    'reg_lambda': (0.1, 30),         # L2 regularization
    'reg_alpha': (0.03, 10),         # L1 regularization
    'scale_pos_weight': (1, 50)      # Adjust for class imbalance
}

# Initialize Bayesian Optimization for LightGBM
optimizer_lgb = BayesianOptimization(
    f=lgb_evaluate, 
    pbounds=pbounds_lgb, 
    random_state=42
)

# Perform optimization
optimizer_lgb.maximize(init_points=5, n_iter=100)

# Extract the best parameters after optimization
best_params_lgb = optimizer_lgb.max['params']
best_params_lgb['n_estimators'] = int(best_params_lgb['n_estimators'])  # Convert float to int
best_params_lgb['max_depth'] = int(best_params_lgb['max_depth'])        # Convert float to int
best_params_lgb['num_leaves'] = int(best_params_lgb['num_leaves'])      # Convert float to int

print(f"Best Parameters for LightGBM: {best_params_lgb}")


In [ ]:
best_params_lgb

In [ ]:
import lightgbm as lgb
model_lgb = lgb.LGBMClassifier(**best_params_lgb)

# Fit the model with early stopping
model_lgb.fit(
    train[significant_features],
    y_train,
    eval_set=[(test[significant_features], y_test)],  # Validation set for early stopping
    eval_metric='binary_logloss',             # Metric for evaluation
)

In [ ]:
test['prob_tuned'] = model_lgb.predict_proba(test[significant_features])[:,1]
oot['prob_tuned'] = model_lgb.predict_proba(oot[significant_features])[:,1]

In [ ]:
best_model

In [ ]:
get_results(test,'prob_tuned','shortcalls_lightgbm_rfe_feat_tuned.xlsx')

In [ ]:
b = oot[oot['prob_tuned']>=0.75].sort_values(by=['date','prob_tuned'],ascending=[True,False])

In [ ]:
a = b.groupby('date').head(2)

In [ ]:
a[['date','nse_symbol','next_day_return_low','target_low_intraday','prob_tuned']].tail(30)

In [ ]:
a['date'] = pd.to_datetime(a['date'])

a['year_month'] = a['date'].dt.to_period('M')

result = a.groupby('year_month').apply(lambda x: pd.Series({
    'target_1_precision': round(x['target_low_intraday'].value_counts(normalize=True).get(1, 0), 2),  # Use built-in round() function
    'tot_nse_symbol': x['nse_symbol'].count(),
    'unique_nse_symbol': x['nse_symbol'].nunique(),
    'unique_days': x['date'].nunique()
})).reset_index()

In [ ]:
result